In [ ]:
import sys
sys.path.append("/Users/mistryk2/Packages/nudobe/src/")
sys.path.append("/Users/mistryk2/Packages/nudobe/")

import numpy as np
import nudobe
from nudobe import EFT, functions, constants
from constants import G_F
from functions import *
from EFT import LEFT
import pandas as pd

In [ ]:
def get_WCs_leptoquark(alpha_SL  = 0,
            alpha_VL  = 0,
            alpha_SR  = 0,
            alpha_VR  = 0,
            epsilon_S = 0,
            epsilon_V = 0, 
            M_V       = 1*TeV, 
            M_S       = 1*TeV
           ):
    WC_LQ = {}
    
    #see arXiv: hep-ph/9603213 for the definition of eps_I, alpha_I
    WC_LQ["SL(6)"] =  1/(G_F*np.sqrt(2)) * epsilon_V/(M_V**2)
    WC_LQ["SR(6)"] =  1/(G_F*np.sqrt(2)) * epsilon_S/(M_S**2)
    WC_LQ["VL(6)"] = +1/(G_F*np.sqrt(2)) * np.sqrt(2) * (alpha_SL/(M_S**2) + alpha_VL/(M_V**2))
    WC_LQ["VR(6)"] = -1/(G_F*np.sqrt(2)) * (alpha_SR/(M_S**2) + alpha_VR/(M_V**2))

    return(WC_LQ)

In [ ]:
KamLAND_limit = 3.6e26

limits = {}    
limits["alpha_SL"]  = np.sqrt(LEFT(get_WCs(alpha_SL  = 1), method = method).t_half("136Xe")/KamLAND_limit)
limits["alpha_VL"]  = np.sqrt(LEFT(get_WCs(alpha_VL  = 1), method = method).t_half("136Xe")/KamLAND_limit)
limits["alpha_SR"]  = np.sqrt(LEFT(get_WCs(alpha_SR  = 1), method = method).t_half("136Xe")/KamLAND_limit)
limits["alpha_VR"]  = np.sqrt(LEFT(get_WCs(alpha_VR  = 1), method = method).t_half("136Xe")/KamLAND_limit)
limits["epsilon_S"] = np.sqrt(LEFT(get_WCs(epsilon_S = 1), method = method).t_half("136Xe")/KamLAND_limit)
limits["epsilon_V"] = np.sqrt(LEFT(get_WCs(epsilon_V = 1), method = method).t_half("136Xe")/KamLAND_limit)